In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
tick = '^GSPC'

stime='2020-01-01'
ftime=None

In [4]:
def get_data_from_ticker(tick, stime, ftime, interval='1d'):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = pd.DataFrame(get_data_from_ticker(tick, stime, ftime))



df.head(3)

,^GSPC
Date,
2020-01-02 00:00:00-05:00,3257.850098
2020-01-03 00:00:00-05:00,3234.850098
2020-01-06 00:00:00-05:00,3246.280029


In [5]:
def add_percent_update(tick):
    """ date stock's or market's update"""
    gf_copy = df.copy(deep=True)
    gf_copy = pd.DataFrame(gf_copy.drop(df.index[0]))
    gf_copy_v = gf_copy[tick].values
    gf_v = df.copy(deep=True)
    gf_vv = gf_v[tick].values

    for i in range(len(gf_copy)):
        gf_vv[i] = (gf_copy_v[i] - gf_vv[i]) / (gf_vv[i] / 100)

    gf_vv = np.insert(gf_vv, 0, 0)
    gf_vv = np.delete(gf_vv, -1)
    return gf_vv

df[f"% update {tick}"] = add_percent_update(tick)
# gf[f"% update {st}"] = add_percent_update(st)

df.head(3)

,^GSPC,% update ^GSPC
Date,,
2020-01-02 00:00:00-05:00,3257.850098,0.000000
2020-01-03 00:00:00-05:00,3234.850098,-0.705987
2020-01-06 00:00:00-05:00,3246.280029,0.353337


Get consistent

In [6]:
deep = 4
type_str = 'vertical_more_or_low'

def analyse_f(df, type_str, deep):
    deep += 1
    # r = list(df.iloc[:,0].values)
    r =  []
    if type_str == 'vertical_more_or_low':
        for i in range(len(df[f'{tick}'])):
            if i <= deep:
                # r[i] = 0
                r.append(0)
            else:    
                l = []
                for y in range(deep):
                    if y == 0: 
                        pass
                    elif y > 0:
                        if df.iloc[i-y, 1] > df.iloc[i-1-y, 1]:
                            x = 'm'
                            l.append(x)
                        else:
                            x = 'l'
                            l.append(x)
                    else:
                        pass
                r.append(l)
    else:
        pass
    return r 
        


df['consistent'] = analyse_f(df, type_str, deep)        
df.tail(5)        

,^GSPC,% update ^GSPC,consistent
Date,,,
2022-12-22 00:00:00-05:00,3822.389893,-1.445170,"[m, m, m, m]"
2022-12-23 00:00:00-05:00,3844.820068,0.586810,"[l, m, m, m]"
2022-12-27 00:00:00-05:00,3829.250000,-0.404962,"[m, l, m, m]"
2022-12-28 00:00:00-05:00,3783.219971,-1.202064,"[l, m, l, m]"
2022-12-29 00:00:00-05:00,3849.280029,1.746133,"[l, l, m, l]"


Поиск наиболее эффективных значений

In [7]:
# df['consistent'] = df['consistent'].astype(str)

In [8]:
''' Переводим в str чтобы дальше можно было сравнить '''
df['consistent'] = df['consistent'].astype(str)

''' Выбираем уникальные '''

df_un = df['consistent'].copy()
df_un = df_un.drop_duplicates()
df_un = df_un.loc[df_un != "0"]
set_consistens = list(df_un)

''' Фильтруем по типам и получаем их суммы '''
r = []
for i in range(len(set_consistens)):
    x = df.loc[(df['consistent'] == set_consistens[i])]
    r.append(f"Сумма {set_consistens[i]} = {x[f'% update {tick}'].sum()}")
r

["Сумма ['m', 'm', 'l', 'm'] = -6.396274967313072",
 "Сумма ['l', 'm', 'm', 'l'] = -3.7077325316189444",
 "Сумма ['m', 'l', 'm', 'm'] = 7.528531635046415",
 "Сумма ['l', 'm', 'l', 'm'] = 5.758008866947426",
 "Сумма ['m', 'l', 'm', 'l'] = 16.260167176230645",
 "Сумма ['l', 'l', 'm', 'm'] = -11.75399030068304",
 "Сумма ['m', 'l', 'l', 'm'] = -32.0119990348091",
 "Сумма ['m', 'm', 'l', 'l'] = -4.72215517547905",
 "Сумма ['l', 'm', 'l', 'l'] = 30.43651322261631",
 "Сумма ['l', 'l', 'l', 'm'] = 5.551431057037231",
 "Сумма ['m', 'l', 'l', 'l'] = -1.9172583405783006",
 "Сумма ['l', 'l', 'm', 'l'] = 15.422332608510326",
 "Сумма ['m', 'm', 'm', 'l'] = 2.6059520196386727",
 "Сумма ['l', 'm', 'm', 'm'] = 9.03132291485174",
 "Сумма ['l', 'l', 'l', 'l'] = -1.2937431458534387",
 "Сумма ['m', 'm', 'm', 'm'] = -4.870531529070221"]

In [9]:
set_consistens

["['m', 'm', 'l', 'm']",
 "['l', 'm', 'm', 'l']",
 "['m', 'l', 'm', 'm']",
 "['l', 'm', 'l', 'm']",
 "['m', 'l', 'm', 'l']",
 "['l', 'l', 'm', 'm']",
 "['m', 'l', 'l', 'm']",
 "['m', 'm', 'l', 'l']",
 "['l', 'm', 'l', 'l']",
 "['l', 'l', 'l', 'm']",
 "['m', 'l', 'l', 'l']",
 "['l', 'l', 'm', 'l']",
 "['m', 'm', 'm', 'l']",
 "['l', 'm', 'm', 'm']",
 "['l', 'l', 'l', 'l']",
 "['m', 'm', 'm', 'm']"]